### 08_Projecting_a_Raster

In [133]:
import gdal
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio import rio
import subprocess
import os

##### Getting the basic information of the raster

In [138]:
!gdalinfo PAN_msk_alt.vrt

Driver: VRT/Virtual Raster
Files: PAN_msk_alt.vrt
       ./PAN_msk_alt.gri
Size is 732, 312
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["unknown",
        SPHEROID["WGS84",6378137,298.257223563]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433]]
Origin = (-83.200000000000003,9.699999999999999)
Pixel Size = (0.008333333000000,-0.008333333000000)
Corner Coordinates:
Upper Left  ( -83.2000000,   9.7000000) ( 83d12' 0.00"W,  9d42' 0.00"N)
Lower Left  ( -83.2000000,   7.1000001) ( 83d12' 0.00"W,  7d 6' 0.00"N)
Upper Right ( -77.1000002,   9.7000000) ( 77d 6' 0.00"W,  9d42' 0.00"N)
Lower Right ( -77.1000002,   7.1000001) ( 77d 6' 0.00"W,  7d 6' 0.00"N)
Center      ( -80.1500001,   8.4000001) ( 80d 9' 0.00"W,  8d24' 0.00"N)
Band 1 Block=732x1 Type=Int16, ColorInterp=Undefined
  Description = PAN_msk_alt
  NoData Value=-9999


##### Converting from vrt to tiff

In [139]:
!gdal_translate "PAN_msk_alt.vrt" "PAN_dem.tif"

Input file size is 732, 312
0...10...20...30...40...50...60...70...80...90...100 - done.


##### Transforming the raster into a map
Calculating the transform information

##### Loading raster dataset

In [140]:
import rasterio
from rasterio import warp
path = 'PAN_dem.tif'
raster = rasterio.open(path)
raster.bounds


BoundingBox(left=-83.2, bottom=7.100000103999999, right=-77.100000244, top=9.7)

##### Calculating the transform for the target coordinate system: EPSG:5472 (Panama-Colon 1911/Panama Polyconic

In [141]:
src_transform = raster.get_transform()

In [142]:
dst_affine, dst_width, dst_height = warp.calculate_default_transform(raster.crs, 5472, raster.width, raster.height,
                                 raster.bounds.left, raster.bounds.bottom, raster.bounds.right,
                                 raster.bounds.top)

In [143]:
dst_affine, dst_width, dst_height

(Affine(1004.3594008532476, 0.0, 734206.6622683606,
        0.0, -1004.3594008532476, 1271031.824124861), 734, 316)

##### Reprojecting the raster dataset

In [144]:
!gdalwarp -overwrite "PAN_dem.tif" "PAN_dem_proj.tif" -s_srs EPSG:4326 -t_srs EPSG:5472 -tr 1000 1000

Creating output file that is 737P x 317L.
Processing PAN_dem.tif [1/1] : 0Using internal nodata values (e.g. -9999) for image PAN_dem.tif.
Copying nodata values from source PAN_dem.tif to destination PAN_dem_proj.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


In [145]:
!gdalinfo "PAN_dem_proj.tif" 

Driver: GTiff/GeoTIFF
Files: PAN_dem_proj.tif
Size is 737, 317
Coordinate System is:
PROJCS["Panama-Colon 1911 / Panama Polyconic",
    GEOGCS["Panama-Colon 1911",
        DATUM["Panama_Colon_1911",
            SPHEROID["Clarke 1866",6378206.4,294.9786982138982,
                AUTHORITY["EPSG","7008"]],
            AUTHORITY["EPSG","1072"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","5467"]],
    PROJECTION["Polyconic"],
    PARAMETER["latitude_of_origin",8.25],
    PARAMETER["central_meridian",-81],
    PARAMETER["false_easting",1000000],
    PARAMETER["false_northing",1092972.1],
    UNIT["Clarke's yard",0.9143917962,
        AUTHORITY["EPSG","9037"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","5472"]]
Origin = (734206.662268360611051,1271031.824124861042947)
Pixel Size = (1000.000000000000000,-1000.000000000000000)
Met